In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, concatenate
from tensorflow.keras.callbacks import EarlyStopping
from Loader import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import datetime

#38.3 s
#8:55 m

In [2]:
BATCH_SIZE = 128

# x_train = None
# x_val = None
x_train = DataLoader(BATCH_SIZE, 'training_generator')
x_val = DataLoader(BATCH_SIZE, 'training_generator', val=True)

In [3]:
lossHistory = []
valHistory = []

class NnueCallbacks(tf.keras.callbacks.Callback):
  def on_batch_end(self, batch, logs=None):
    return
    lossHistory[-1].append(logs['loss'])
    if batch and batch % 15000 == 0:

      pred = self.model.predict(x_val)
      loss = (pred.flatten() - x_val.labels.flatten()[0:len(x_val)*BATCH_SIZE]) ** 2
      loss = np.sum(loss) / len(loss)
      valHistory[-1].append(loss)

      for i in lossHistory:
        plt.plot(i[20:], linewidth=0.5)
        plt.savefig("loss.png", dpi=400)
      plt.clf()

      for i in valHistory:
        plt.plot(i, linewidth=0.5)
        plt.savefig("val.png", dpi=400)
      
      plt.clf()

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [7]:
from keras import backend as K
def clipped_relu(x):
    return K.relu(x, max_value=4)

input1 = Input(shape=(768,), sparse=True)
input2 = Input(shape=(768,), sparse=True)
accumulator = Dense(128, activation=clipped_relu)

layers = concatenate([accumulator(input1), accumulator(input2)])
layers = Dense(32, activation=clipped_relu)(layers)
layers = Dense(32, activation=clipped_relu)(layers)
layers = Dense(1, activation='sigmoid')(layers)

model = Model(inputs=[input1, input2], outputs=[layers])

# model = Sequential()
# model.add(Dense(128, input_shape=(769,), activation=clipped_relu))
# model.add(Dense(32, activation=clipped_relu))
# model.add(Dense(32, activation=clipped_relu))
# model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [ ]:
model = tf.keras.models.load_model("production/")

In [8]:
lossHistory.append([])
valHistory.append([])
model.fit(x_train, validation_data=x_val, epochs=100, callbacks=[NnueCallbacks(), EarlyStopping(patience=1)])

# Epoch 1/100
# 959458/959458 [==============================] - 1610s 2ms/step - loss: 0.0170 - mae: 0.0809 - val_loss: 0.0155 - val_mae: 0.0768
# Epoch 2/100
# 959458/959458 [==============================] - 1603s 2ms/step - loss: 0.0161 - mae: 0.0779 - val_loss: 0.0152 - val_mae: 0.0747
# Epoch 3/100
# 959458/959458 [==============================] - 1601s 2ms/step - loss: 0.0159 - mae: 0.0773 - val_loss: 0.0150 - val_mae: 0.0750
# Epoch 4/100
# 959458/959458 [==============================] - 1597s 2ms/step - loss: 0.0158 - mae: 0.0770 - val_loss: 0.0150 - val_mae: 0.0746
# Epoch 5/100
# 959458/959458 [==============================] - 1601s 2ms/step - loss: 0.0158 - mae: 0.0768 - val_loss: 0.0150 - val_mae: 0.0744
# Epoch 6/100
# 959458/959458 [==============================] - 1600s 2ms/step - loss: 0.0157 - mae: 0.0767 - val_loss: 0.0149 - val_mae: 0.0744
# Epoch 7/100
# 959458/959458 [==============================] - 1607s 2ms/step - loss: 0.0157 - mae: 0.0767 - val_loss: 0.0150 - val_mae: 0.0751

Epoch 1/100
959458/959458 [==============================] - 1881s 2ms/step - loss: 0.0164 - mae: 0.0788 - val_loss: 0.0150 - val_mae: 0.0742
Epoch 2/100
959458/959458 [==============================] - 1870s 2ms/step - loss: 0.0156 - mae: 0.0761 - val_loss: 0.0147 - val_mae: 0.0735
Epoch 3/100
959458/959458 [==============================] - 1920s 2ms/step - loss: 0.0154 - mae: 0.0756 - val_loss: 0.0146 - val_mae: 0.0735
Epoch 4/100
959458/959458 [==============================] - 1911s 2ms/step - loss: 0.0154 - mae: 0.0754 - val_loss: 0.0146 - val_mae: 0.0730
Epoch 5/100
959458/959458 [==============================] - 1900s 2ms/step - loss: 0.0154 - mae: 0.0753 - val_loss: 0.0145 - val_mae: 0.0731
Epoch 6/100
959458/959458 [==============================] - 1892s 2ms/step - loss: 0.0153 - mae: 0.0752 - val_loss: 0.0146 - val_mae: 0.0729


In [7]:
import Features
import numpy as np

FEN = "r3n1k1/2qbbppp/4p3/3p2P1/2p2P2/2N1P3/Pr1B2BP/R2Q1RK1 b - - 2 19"
features = Features.get(FEN)
model.predict(np.array([features]))

array([[0.15982483]], dtype=float32)

In [9]:
model.save("production")

INFO:tensorflow:Assets written to: production/assets


INFO:tensorflow:Assets written to: production/assets
